<a href="https://colab.research.google.com/github/hamamh66/Explainable-Multi-Agent-Energy-Twin-for-Resilient-and-Carbon-Aware-Grid-Scheduling/blob/main/QFF_Improved_Federated_VQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum-Federated Forecasting (QFF) with VQC-Inspired Encoding

Clean, documented notebook for:

- Loading ISO-NE hourly data (2024)
- Building supervised 168→24 forecasting windows
- Training centralized baselines: LSTM, TCN, Transformer
- Training a federated QFF model (VQC-inspired encoder + LSTM head)
- Evaluating MAE, RMSE, MAPE on the test set
- Plotting seasonal error profiles
- Plotting capacity-margin curves for a representative 24 h horizon

**Note:**
- This notebook assumes that the CSV file `2024_smd_hourly.csv` is already in your Google Drive.
- You may need to adapt the column names in the data loading cell to match your file.


In [ ]:
# Install / import all required libraries

!pip install tensorflow-addons -q

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)


In [ ]:
# (Optional) Mount Google Drive when running in Colab

from google.colab import drive
drive.mount('/content/drive')

# Base directory in your Drive where data and figures will be saved
BASE_DIR = "/content/drive/MyDrive/QFF/"  # adapt if needed
os.makedirs(BASE_DIR, exist_ok=True)

# Path to ISO-NE hourly CSV (already downloaded from ISO-NE)
DATA_PATH = os.path.join(BASE_DIR, "2024_smd_hourly.csv")  # adapt file name if needed
print("Data path:", DATA_PATH)


## 1. Data Loading and Preprocessing

We assume an hourly time series for one zone (or aggregated system):

- A timestamp (local or UTC)
- A target column (e.g., load or price)
- Optionally zone/aggregation columns

Adapt the column names in the next cell to match your CSV.


In [ ]:
# Load ISO-NE hourly data and build a single time series

df = pd.read_csv(DATA_PATH)
print("Raw columns:\n", df.columns.tolist())
print("First 5 rows:")
display(df.head())

# === ADAPT THIS BLOCK AS NEEDED ===
# Try to identify a timestamp column and a numerical target column.
# Example patterns (uncomment / adapt depending on your CSV):

# 1) If you already have a combined datetime column, e.g. 'Datetime':
# df["timestamp"] = pd.to_datetime(df["Datetime"])

# 2) If you have 'Local Date' and 'Local Hour Ending' columns:
# df["timestamp"] = pd.to_datetime(df["Local Date"] + " " + df["Local Hour Ending"].astype(str) + ":00")

# 3) Generic fallback: try to infer datetime from a column named 'Date' or 'Date/Time':
if "timestamp" not in df.columns:
    for cand in ["Date/Time", "Datetime", "DateTime", "Date", "DATE"]:
        if cand in df.columns:
            df["timestamp"] = pd.to_datetime(df[cand])
            break

if "timestamp" not in df.columns:
    raise ValueError("Please define df['timestamp'] from your date/hour columns.")

    # Choose a numeric target column (e.g., system load or price). Replace this with your actual column.
# Examples: 'RT_Demand_MW', 'DA_Demand_MW', 'Load', 'VALUE'.
TARGET_COL = "Load"  # TODO: adapt to your actual target column name
if TARGET_COL not in df.columns:
    raise ValueError(f"Column '{TARGET_COL}' not found. Please set TARGET_COL to a valid numeric column.")

# Keep only timestamp and target for now
df = df[["timestamp", TARGET_COL]].copy()
df = df.sort_values("timestamp").reset_index(drop=True)

print("Data after selection:")
display(df.head())
print("Number of rows:", len(df))


### 1.1 Feature Engineering

We derive simple calendar features from the timestamp:

- Hour of day
- Day of week
- Month of year
- (Optionally) day of year


In [ ]:
# Build calendar features

df["hour"] = df["timestamp"].dt.hour
df["dow"] = df["timestamp"].dt.dayofweek  # Monday=0
df["month"] = df["timestamp"].dt.month
df["doy"] = df["timestamp"].dt.dayofyear

# Reorder columns: timestamp, target, then features
cols = ["timestamp", TARGET_COL, "hour", "dow", "month", "doy"]
df = df[cols]

display(df.head())


### 1.2 Scaling and Supervised Windowing (168→24)

We convert the time series into supervised learning windows:

- Input window length: 168 hours (7 days)
- Forecast horizon: 24 hours ahead

Features are standardized; the target is also scaled but later inverse-transformed for metrics and plots.


In [ ]:
LOOKBACK_H = 168   # input window size
HORIZON_H  = 24    # output horizon

# Feature matrix (excluding timestamp)
feature_cols = [TARGET_COL, "hour", "dow", "month", "doy"]
X_raw = df[feature_cols].values.astype(np.float32)
y_raw = df[TARGET_COL].values.astype(np.float32)

# Scale features and target separately
x_scaler = StandardScaler()
X_scaled = x_scaler.fit_transform(X_raw)

y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y_raw.reshape(-1, 1)).ravel()

timestamps = df["timestamp"].values

def build_supervised_sequences(X, y, ts, lookback=168, horizon=24):
    """Build (X_seq, Y_seq, TS_targets) for sliding-window forecasting."""
    X_seqs = []
    Y_seqs = []
    TS_targets = []

    n = len(y)
    max_start = n - lookback - horizon + 1
    for start in range(max_start):
        end_in = start + lookback
        end_out = end_in + horizon

        X_win = X[start:end_in]
        Y_win = y[end_in:end_out]
        TS_win = ts[end_in:end_out]

        X_seqs.append(X_win)
        Y_seqs.append(Y_win)
        TS_targets.append(TS_win)

    return (
        np.array(X_seqs, dtype=np.float32),
        np.array(Y_seqs, dtype=np.float32),
        np.array(TS_targets)
    )

X_seq, Y_seq, TS_targets = build_supervised_sequences(
    X_scaled, y_scaled, timestamps,
    lookback=LOOKBACK_H, horizon=HORIZON_H
)

print("X_seq shape:", X_seq.shape)      # (n_samples, 168, n_features)
print("Y_seq shape:", Y_seq.shape)      # (n_samples, 24)
print("TS_targets shape:", TS_targets.shape)  # (n_samples, 24)


### 1.3 Train/Validation/Test Split

We perform a chronological split of the supervised sequences. For example:

- 70% training
- 15% validation
- 15% testing


In [ ]:
n_samples = X_seq.shape[0]
train_end = int(0.7 * n_samples)
val_end   = int(0.85 * n_samples)

X_train, Y_train, TS_train = X_seq[:train_end], Y_seq[:train_end], TS_targets[:train_end]
X_val,   Y_val,   TS_val   = X_seq[train_end:val_end], Y_seq[train_end:val_end], TS_targets[train_end:val_end]
X_test,  Y_test,  TS_test  = X_seq[val_end:], Y_seq[val_end:], TS_targets[val_end:]

print("Train shape:", X_train.shape, Y_train.shape)
print("Val shape:  ", X_val.shape,   Y_val.shape)
print("Test shape: ", X_test.shape,  Y_test.shape)

n_features = X_train.shape[2]
input_len  = LOOKBACK_H
output_len = HORIZON_H


## 2. Centralized Baseline Models

We define three baseline architectures:

- LSTM-based sequence model
- TCN (Temporal Convolutional Network)
- Transformer-style encoder

All models take an input of shape `(lookback, n_features)` and output a vector of length `HORIZON_H`.


In [ ]:
def build_lstm_model(input_len, n_features, output_len):
    inputs = keras.Input(shape=(input_len, n_features))
    x = layers.LSTM(64, return_sequences=True)(inputs)
    x = layers.LSTM(32)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(output_len)(x)
    model = keras.Model(inputs, outputs, name="lstm_baseline")
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="mse"
    )
    return model

def build_tcn_model(input_len, n_features, output_len):
    inputs = keras.Input(shape=(input_len, n_features))
    x = layers.Conv1D(64, kernel_size=3, padding="causal", activation="relu")(inputs)
    x = layers.Conv1D(64, kernel_size=3, padding="causal", activation="relu")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(output_len)(x)
    model = keras.Model(inputs, outputs, name="tcn_baseline")
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="mse"
    )
    return model

def build_transformer_encoder(input_len, n_features, d_model=64, num_heads=2, ff_dim=128, num_layers=2):
    """Simple Transformer encoder stack returning a sequence output."""
    inputs = keras.Input(shape=(input_len, n_features))
    # Linear projection to d_model
    x = layers.Dense(d_model)(inputs)

    for _ in range(num_layers):
        # Multi-head self-attention block
        attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

        # Feed-forward block
        ff = layers.Dense(ff_dim, activation="relu")(x)
        ff = layers.Dense(d_model)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    return inputs, x

def build_transformer_model(input_len, n_features, output_len):
    inputs, encoded = build_transformer_encoder(input_len, n_features)
    x = layers.GlobalAveragePooling1D()(encoded)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(output_len)(x)
    model = keras.Model(inputs, outputs, name="transformer_baseline")
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="mse"
    )
    return model

print("Models defined.")


### 2.1 Training Centralized Baselines

We train each baseline for a moderate number of epochs with early stopping on the validation loss.


In [ ]:
EPOCHS = 20
BATCH_SIZE = 64

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

print("Training centralized LSTM...")
lstm_model = build_lstm_model(input_len, n_features, output_len)
history_lstm = lstm_model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop],
    verbose=1
)

print("Training centralized TCN...")
tcn_model = build_tcn_model(input_len, n_features, output_len)
history_tcn = tcn_model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop],
    verbose=1
)

print("Training centralized Transformer...")
transformer_model = build_transformer_model(input_len, n_features, output_len)
history_tr = transformer_model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop],
    verbose=1
)

print("Baseline training complete.")


## 3. Federated QFF Model with VQC-Inspired Encoding

We now define the QFF model:

- Input: `(lookback, n_features)`
- VQC-inspired encoder:
  - Time-distributed dense projection to a higher-dimensional latent space ("parameterized rotations")
  - Sine/cosine mixing to emulate interference-like behavior
- LSTM head for temporal forecasting
- Output: 24-step horizon

Federated learning is simulated by partitioning the training data into `K` nodes and performing a simple FedAvg loop over `R` rounds.


In [ ]:
def vqc_encoder_block(inputs, d_model=64, depth=2):
    """Simple VQC-inspired encoder using TimeDistributed dense + sine/cosine mixing."""
    x = inputs
    for _ in range(depth):
        # Parameterized linear map
        x_lin = layers.TimeDistributed(layers.Dense(d_model))(x)
        # Nonlinear phase-like transformation
        x_sin = tf.sin(x_lin)
        x_cos = tf.cos(x_lin)
        # Concatenate and project back to d_model
        x = layers.Concatenate(axis=-1)([x_sin, x_cos])
        x = layers.TimeDistributed(layers.Dense(d_model, activation="relu"))(x)
    return x

def build_qff_vqc_model(input_len, n_features, output_len, d_model=64, depth=2):
    inputs = keras.Input(shape=(input_len, n_features))
    # VQC-inspired encoder
    z = vqc_encoder_block(inputs, d_model=d_model, depth=depth)
    # LSTM head
    h = layers.LSTM(64, return_sequences=True)(z)
    h = layers.LSTM(32)(h)
    h = layers.Dense(64, activation="relu")(h)
    outputs = layers.Dense(output_len)(h)
    model = keras.Model(inputs, outputs, name="qff_vqc_model")
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="mse"
    )
    return model

print("QFF model definition ready.")


### 3.1 Simple Federated Averaging (FedAvg) Simulation

We split the training data into `K` nodes and perform `R` federated rounds:

For each round:
1. Broadcast global weights to each node model.
2. Train locally on each node for a small number of epochs.
3. Collect updated weights from each node.
4. Average weights (FedAvg) to obtain the new global model.


In [ ]:
def get_federated_splits(X, Y, K=3):
    """Split (X, Y) along sample dimension into K shards for federated nodes."""
    n = X.shape[0]
    splits_X = np.array_split(X, K, axis=0)
    splits_Y = np.array_split(Y, K, axis=0)
    return list(zip(splits_X, splits_Y))

def average_weights(weight_list):
    """Average a list of weight lists (FedAvg)."""
    avg = []
    for weights in zip(*weight_list):
        w_stack = np.stack(weights, axis=0)
        avg.append(np.mean(w_stack, axis=0))
    return avg

# Federated configuration
K_NODES = 3
R_ROUNDS = 5
LOCAL_EPOCHS = 1
LOCAL_BATCH = 64

fed_splits = get_federated_splits(X_train, Y_train, K=K_NODES)
for i, (Xk, Yk) in enumerate(fed_splits):
    print(f"Node {i}: X shape = {Xk.shape}, Y shape = {Yk.shape}")

print("Training federated QFF (VQC-inspired)...")
qff_model_global = build_qff_vqc_model(input_len, n_features, output_len, d_model=64, depth=2)

for r in range(R_ROUNDS):
    print(f"\n[Round {r+1}/{R_ROUNDS}]")
    node_weights = []
    for node_id, (Xk, Yk) in enumerate(fed_splits):
        print(f"  Node {node_id}: local training...")
        # Clone global model for local node
        node_model = build_qff_vqc_model(input_len, n_features, output_len, d_model=64, depth=2)
        node_model.set_weights(qff_model_global.get_weights())

        node_model.fit(
            Xk, Yk,
            epochs=LOCAL_EPOCHS,
            batch_size=LOCAL_BATCH,
            verbose=0
        )

        node_weights.append(node_model.get_weights())

    # FedAvg
    new_global_weights = average_weights(node_weights)
    qff_model_global.set_weights(new_global_weights)

print("Federated QFF training complete.")


## 4. Evaluation on Test Set

We compute MAE, RMSE, and MAPE on the **inverse-scaled** test predictions for all models.


In [ ]:
def mae(y_true, y_pred):
    return float(mean_absolute_error(y_true, y_pred))

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def mape(y_true, y_pred):
    eps = 1e-6
    return float(np.mean(np.abs((y_true - y_pred) / (y_true + eps))) * 100.0)

def inverse_targets(y_scaled_seq):
    flat = y_scaled_seq.reshape(-1, 1)
    inv_flat = y_scaler.inverse_transform(flat).ravel()
    return inv_flat.reshape(y_scaled_seq.shape)

# 1) Get scaled predictions on the test set
Y_pred_lstm_scaled = lstm_model.predict(X_test, verbose=0)
Y_pred_tcn_scaled  = tcn_model.predict(X_test, verbose=0)
Y_pred_tr_scaled   = transformer_model.predict(X_test, verbose=0)
Y_pred_qff_scaled  = qff_model_global.predict(X_test, verbose=0)

# 2) Inverse scale
Y_test_inv      = inverse_targets(Y_test)
Y_pred_lstm_inv = inverse_targets(Y_pred_lstm_scaled)
Y_pred_tcn_inv  = inverse_targets(Y_pred_tcn_scaled)
Y_pred_tr_inv   = inverse_targets(Y_pred_tr_scaled)
Y_pred_qff_inv  = inverse_targets(Y_pred_qff_scaled)

# 3) Flatten for global metrics
Y_true_flat = Y_test_inv.reshape(-1)
Y_lstm_flat = Y_pred_lstm_inv.reshape(-1)
Y_tcn_flat  = Y_pred_tcn_inv.reshape(-1)
Y_tr_flat   = Y_pred_tr_inv.reshape(-1)
Y_qff_flat  = Y_pred_qff_inv.reshape(-1)

metrics = {}
for name, yhat in [
    ("QFF", Y_qff_flat),
    ("Transformer", Y_tr_flat),
    ("LSTM", Y_lstm_flat),
    ("TCN", Y_tcn_flat),
]:
    metrics[name] = {
        "MAE": mae(Y_true_flat, yhat),
        "RMSE": rmse(Y_true_flat, yhat),
        "MAPE": mape(Y_true_flat, yhat),
        "CRPS": None
    }

print("Test metrics (inverse-scaled):")
for name, m in metrics.items():
    print(
        f"{name}: MAE={m['MAE']:.3f}, RMSE={m['RMSE']:.3f}, MAPE={m['MAPE']:.3f}"
    )


## 5. Seasonal Error Profiles (Full Year)

We now compute seasonal MAE (Winter, Spring, Summer, Fall) over the *entire* test horizon and produce a grouped bar chart.


In [ ]:
def get_season(dt):
    """Simple season mapping for Northern Hemisphere."""
    m = dt.month
    if m in [12, 1, 2]:
        return "Winter"
    elif m in [3, 4, 5]:
        return "Spring"
    elif m in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

def seasonal_mae_flat(y_true_flat, y_pred_flat, ts_flat):
    seasons = pd.Series(ts_flat).apply(get_season)
    df_tmp = pd.DataFrame({
        "y": y_true_flat,
        "yhat": y_pred_flat,
        "season": seasons.values
    })
    out = df_tmp.groupby("season").apply(
        lambda g: mean_absolute_error(g["y"], g["yhat"])
    )
    return out.reindex(["Winter", "Spring", "Summer", "Fall"])

# Flatten timestamps of the test targets
TS_test_flat = TS_test.reshape(-1)

mae_qff  = seasonal_mae_flat(Y_true_flat, Y_qff_flat, TS_test_flat)
mae_lstm = seasonal_mae_flat(Y_true_flat, Y_lstm_flat, TS_test_flat)
mae_tcn  = seasonal_mae_flat(Y_true_flat, Y_tcn_flat,  TS_test_flat)
mae_tr   = seasonal_mae_flat(Y_true_flat, Y_tr_flat,   TS_test_flat)

print("Seasonal MAE (MW) on test set:")
print("QFF:\n", mae_qff)
print("LSTM:\n", mae_lstm)
print("TCN:\n", mae_tcn)
print("Transformer:\n", mae_tr)

models = ["QFF", "LSTM", "TCN", "Transformer"]
seasons_order = ["Winter", "Spring", "Summer", "Fall"]
mae_values = np.array([
    mae_qff.values,
    mae_lstm.values,
    mae_tcn.values,
    mae_tr.values
])

x = np.arange(len(seasons_order))
width = 0.2

plt.figure(figsize=(10, 6))
for i, model in enumerate(models):
    plt.bar(x + i*width, mae_values[i], width=width, label=model)

plt.xticks(x + 1.5*width, seasons_order)
plt.ylabel("MAE (MW)")
plt.title("Seasonal Error Profiles — QFF vs Centralized Baselines (Test Set)")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.legend()

fig1_path = os.path.join(BASE_DIR, "fig_seasonal_error_profiles.pdf")
plt.savefig(fig1_path, bbox_inches="tight")
plt.close()

print("Saved seasonal error profile figure to:", fig1_path)


## 6. Capacity-Margin Curves for a Representative 24-Hour Horizon

We define a simple capacity margin model:

- Choose one test sample (24-hour horizon)
- Use a constant capacity `C_t = C` chosen slightly above the observed maximum for that day
- Compute true margin `M_t = C_t - y_t` and QFF-based margin `\hat{M}_t = C_t - \hat{y}_t`
- Plot both curves to visualize under/over-estimation patterns


In [ ]:
# Select a representative 24-hour horizon from the test set
idx_sample = 0  # change to inspect another day

y_true_day = Y_test_inv[idx_sample]       # shape (24,)
y_qff_day  = Y_pred_qff_inv[idx_sample]   # shape (24,)
ts_day     = TS_test[idx_sample]          # array of datetimes (24,)

# Define a simple capacity level slightly above the daily max
C_level = float(y_true_day.max() + 0.10 * (y_true_day.max() - y_true_day.min()))
C_day = np.full_like(y_true_day, C_level)

M_true = C_day - y_true_day
M_qff  = C_day - y_qff_day

plt.figure(figsize=(10, 5))
plt.plot(ts_day, M_true, label="True margin", linewidth=2)
plt.plot(ts_day, M_qff, label="QFF-based margin", linestyle="--", linewidth=2)
plt.xticks(rotation=45)
plt.ylabel("Capacity margin (MW)")
plt.title("True vs QFF-based Capacity Margins (Representative 24 h)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()

fig2_path = os.path.join(BASE_DIR, "fig_capacity_margin_curves.pdf")
plt.savefig(fig2_path, bbox_inches="tight")
plt.close()

print("Saved capacity-margin figure to:", fig2_path)


## 7. Summary

- The notebook builds a corrected 168→24 supervised pipeline for ISO-NE hourly data.
- Centralized baselines (LSTM, TCN, Transformer) and a federated QFF model are trained.
- Global MAE/RMSE/MAPE are reported on the inverse-scaled test set.
- Seasonal MAE profiles and capacity-margin curves are generated as PDF figures in `BASE_DIR`.

These results can be directly used to populate the quantitative comparison table and figures in the manuscript.
